# QuickGuide for Image analysis
[프로젝트 페이지: Research-on-the-TV-market](https://github.com/xikest/research-market-tv)

In [ ]:
!pip install -U pandas openpyxl tqdm
!pip install -U requests selenium beautifulsoup4 PyMuPDF
!pip install -U wordcloud nltk 
!pip install -U scikit-learn openai 
!pip install -U matplotlib seaborn
!pip install -U opencv-python pytube 
!pip install -U mkt-retv

## Env 세팅

In [1]:
from market_research.analysis import ImgAnalysis

## 준비

### 분석 폴더
- `input`에 파일을 넣어주세요.

In [2]:
ima = ImgAnalysis()

## Video 분석

#### 비디오 다운로드 from yt

In [ ]:
ima.preset_video_dict

In [ ]:
for title, url in ima.preset_video_dict.items():
    video_path= ima.download_video_from_yt(url)

In [ ]:
video_paths = ima.read_files_from_inputpath(docs_type='video')
video_paths

#### video 분석

In [ ]:
for video_path in video_paths:
    ima.process_video(video_path,frame_interval=60, showmode=False)

## Image 분석

In [5]:
image_list = ima.read_files_from_inputpath()
# image_list

In [ ]:
ima.to_lab_images(image_list, showmode=True)

## colab에서 파일 받기

In [ ]:
from google.colab import files
!zip -r /content/results.zip /content/results
files.download('/content/results.zip')

---